In [ ]:
import json
import string
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import morfeusz2
import itertools as it
import operator as op
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import LatentDirichletAllocation as LDA, NMF
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


### Importy

1. Utworzenie korpusu dokumentów

In [ ]:
corpus_dir = "./literature" 
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
files = corpus.fileids()
files

2.  Wstępne przetworzenie dokumentów

In [ ]:
documents = {}
for f in files:
    documents[f] = corpus.raw(f)
print(json.dumps(documents, indent=4, ensure_ascii=False))


In [ ]:
stoplist_file = open('stopwords_pl.txt', 'r', encoding='UTF-8')
stoplist = stoplist_file.read().splitlines()
stoplist_file.close()
stoplist = stoplist[4:]
stoplist

In [ ]:
def lemmatize(text):
    morf = morfeusz2.Morfeusz()
    segments = it.groupby(morf.analyse(text), op.itemgetter(0,1))

    def disambiguate(group):
        pairs = ((len(descr), lemma)
                for _, _, (_, lemma, descr, _, _,) in group)
        perpl, lemma = min(pairs)
        return lemma.split(":")[0]
    
    lemmas = (disambiguate(group) for key, group in segments)
    return " ".join(filter(str.isalpha, lemmas))


In [ ]:
for key in documents:
    documents[key] = documents[key].lower()
    documents[key] = "".join([char for char in documents[key] if char not in string.punctuation])
    documents[key] = lemmatize(documents[key])
    documents[key] = " ".join([word for word in word_tokenize(documents[key], language='polish') if word not in stoplist])
print(json.dumps(documents, indent=4, ensure_ascii=False))

3. Utworzenie macierzy częstości

In [ ]:
docs = pd.DataFrame.from_dict(documents, orient='index', columns=['content'])
docs

In [ ]:
count_vectorizer = CountVectorizer()
count_tf = count_vectorizer.fit_transform(docs['content'])
count_tf.toarray()

In [ ]:
tfid_vectorizer = TfidfVectorizer()
count_tfid = tfid_vectorizer.fit_transform(docs['content'])
count_tfid.toarray()

4. Chmury tagów

In [ ]:
cloud = WordCloud(width=800, height=400, background_color='white', max_words=5000, contour_color='steelblue', contour_width=3).generate_from_frequencies(count_vectorizer.vocabulary_)
if not os.path.exists('./wordclouds'):
    os.makedirs('./wordclouds')
for index, row in docs.iterrows():
    cloud.generate(row['content'])
    plt.axis("off")
    title = os.path.basename(index).replace('.txt', '')
    plt.imshow(cloud)
    plt.title(title)
    plt.savefig(f'./wordclouds/{title}.png', bbox_inches='tight')

5. Topic modelling

In [ ]:
if not os.path.exists("./tematy"):
    os.mkdir("./tematy")
def plot_top_words(model, feature_names, n_top_words, title, subplots):
    fig, axes = plt.subplots(*subplots, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.savefig(f"./tematy/{title}.png")

In [ ]:
def plot_documents(model, counts, files_names, n_components, title):
  colors = ['forestgreen', 'lightskyblue', 'hotpink', 'turquoise', 'steelblue', 'crimson', 'seagreen', 'orange']
  docs_topics = pd.DataFrame(model.transform(counts), columns=[f"Topic {x}" for x in range(n_components)])
  docs_topics.index = [file_name.replace(".txt", "") for file_name in files_names]
  plt.figure(figsize=(7,4))
  left = [0] * len(docs_topics)
  for i, col in enumerate(docs_topics.columns):
    plt.barh(docs_topics.index, docs_topics[col], left=left, label=col, color = colors[i])
    left = [left[j]+docs_topics[col].iloc[j] for j in range(len(docs_topics))]
    plt.legend(loc='upper right', fontsize=12)
  plt.savefig(f"./tematy/{title}_docs.png")
  plt.close()

In [ ]:
n_components = 6
feature_names = count_vectorizer.get_feature_names_out()
n_top_words = 20
subplots = (2,3)

In [ ]:
lda = LDA(n_components=n_components,  max_iter=5, learning_method='online', learning_offset=50, random_state=0)
lda.fit(count_tf)
plot_top_words(lda, feature_names, n_top_words, 'Tematy w modelu LDA', subplots)
plot_documents(lda, count_tf, files, n_components, 'LDA')

In [ ]:
nmf_fn = NMF(
    n_components=n_components,
    random_state=1,
    alpha_H=.00005,
    alpha_W=.00005,
    l1_ratio=.5
)
nmf_fn.fit(count_tfid)
plot_top_words(nmf_fn, feature_names, n_top_words, "Tematy w modelu NMF (FN)", subplots)
plot_documents(nmf_fn, count_tf, files, n_components, 'NMF_FN')


In [ ]:
nmf_kl = NMF(
    n_components=n_components,
    random_state=1,
    beta_loss='kullback-leibler',
    solver='mu',
    max_iter=1000,
    alpha_H=.00005,
    alpha_W=.00005,
    l1_ratio=.5
)
nmf_kl.fit(count_tfid)
plot_top_words(nmf_kl, feature_names, n_top_words, "Tematy w modelu NMF (KL)", subplots)
plot_documents(nmf_kl, count_tf, files, n_components, 'NMF_KL')



6. Analiza skupień

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import dendrogram

from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    dendrogram(linkage_matrix, **kwargs)

In [ ]:
cs = cosine_similarity(count_tfid, count_tfid).flatten().reshape(20,20)
cs

In [ ]:
clustering = AgglomerativeClustering(
    n_clusters=6, metric='euclidean', linkage='ward', compute_distances=True
).fit(cs)
clustering.labels_

In [ ]:
plot_dendrogram(clustering, labels=files, leaf_rotation=90, leaf_font_size=12)

In [ ]:
ed = euclidean_distances(count_tf, count_tf).flatten().reshape(20,20)
ed

In [ ]:
clustering_e = AgglomerativeClustering(
    n_clusters=6, metric='precomputed', linkage='complete', compute_distances=True
).fit(ed)
clustering_e.labels_

In [ ]:
plot_dendrogram(clustering_e, labels=files, leaf_rotation=90, leaf_font_size=12)

7. N-gramy

In [ ]:
documents_tokenized = {}
for key in documents:
    documents_tokenized[key] = word_tokenize(documents[key], language='polish')
print(json.dumps(documents_tokenized, indent=4, ensure_ascii=False))

In [ ]:
for title in documents_tokenized:
    n_gram = pd.Series(ngrams(documents_tokenized[title], n_count)).value_counts()
    plt.figure(figsize=(10, 6))
    n_gram[:5].plot.barh()
    save_path = os.path.join(ngram_dir, title.replace('.txt', '.png'))
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

In [ ]:
texts = " ".join(docs['content']).split(" ")
n_gram = pd.Series(ngrams(texts, 3)).value_counts()
n_gram[:15].plot.barh()
plt.gcf().subplots_adjust(left=0.4)
plt.savefig("./ngramy/all.png")
plt.close()